In [4]:
import pyblp
import numpy as np
import pandas as pd
import statsmodels.api as sm

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

# Load the data 
sales_data = pd.read_csv("OTC_Sales.csv", header=0, sep='\t')
print(sales_data.head())

demo_data = pd.read_csv("OTC_Demographics.csv")

   store  week  brand  sales_  count  price_  prom_  cost_
0      2     1      1      16  14181    3.29    0.0   2.06
1      2     2      1      12  13965    3.27    0.0   2.04
2      2     3      1       6  13538    3.37    0.0   2.15
3      2     4      1      12  13735    3.30    0.0   2.07
4      2     5      1      10  13735    3.34    0.0   2.12


In [5]:
#QUESTION 1
#Now, well start the calculations for the Sum Stats Table. 
#Market Shares 
#Note- for this table I want the market shares to be meaningful in terms of one another (headache medicine). 
# total sales per store-week 
sales_data["total_sales"] = sales_data.groupby(["store", "week"])["sales_"].transform("sum")

# market share: sales of product i / total sales in the store-week
sales_data["market_share"] = sales_data["sales_"] / sales_data["total_sales"]

# Calculate market share for each product (brand)
market_share = sales_data.groupby("brand")["market_share"].mean()

# Print the market share for each product
for brand, share in market_share.items():
    print(f"Market Share of Product_{brand}: {share:.2%}")


Market Share of Product_1: 14.88%
Market Share of Product_2: 17.76%
Market Share of Product_3: 11.65%
Market Share of Product_4: 12.01%
Market Share of Product_5: 7.71%
Market Share of Product_6: 3.60%
Market Share of Product_7: 4.33%
Market Share of Product_8: 3.47%
Market Share of Product_9: 8.16%
Market Share of Product_10: 9.30%
Market Share of Product_11: 7.13%


In [6]:
#Unit Price (price per 1 bottle of a product)
unit_price = sales_data.groupby("brand")["price_"].mean()

# average unit price for each product 
for product, price in unit_price.items():
    print(f"Average Unit Price of Product_{product}: {price:.2f}")


Average Unit Price of Product_1: 3.42
Average Unit Price of Product_2: 4.94
Average Unit Price of Product_3: 7.02
Average Unit Price of Product_4: 2.96
Average Unit Price of Product_5: 5.15
Average Unit Price of Product_6: 8.16
Average Unit Price of Product_7: 2.67
Average Unit Price of Product_8: 3.61
Average Unit Price of Product_9: 3.97
Average Unit Price of Product_10: 1.93
Average Unit Price of Product_11: 4.45


In [7]:
#price per 100 tablets 
price_per_100 = {}

# Tablets per bottle for each product 
tablets_per_bottle = {
    1: 25,  # Tylenol 25
    2: 50,  # Tylenol 50
    3: 100,  # Tylenol 100
    4: 25,  # Advil 25
    5: 50,  # Advil 50
    6: 100,  # Advil 100
    7: 25,  # Bayer 25
    8: 50,  # Bayer 50
    9: 100,  # Bayer 100
    10: 50,  # Store Brand 50
    11: 100  # Store Brand 100
}

# Calculate price per 100 tablets based on the number of tablets per bottle
for product, price in unit_price.items():
    if tablets_per_bottle[product] == 25:
        price_per_100[product] = price * 4  
    elif tablets_per_bottle[product] == 50:
        price_per_100[product] = price * 2  
    elif tablets_per_bottle[product] == 100:
        price_per_100[product] = price  

for product, price in price_per_100.items():
    print(f"Price per 100 tablets of Product_{product}: {price:.2f}")


Price per 100 tablets of Product_1: 13.68
Price per 100 tablets of Product_2: 9.88
Price per 100 tablets of Product_3: 7.02
Price per 100 tablets of Product_4: 11.85
Price per 100 tablets of Product_5: 10.29
Price per 100 tablets of Product_6: 8.16
Price per 100 tablets of Product_7: 10.69
Price per 100 tablets of Product_8: 7.21
Price per 100 tablets of Product_9: 3.97
Price per 100 tablets of Product_10: 3.86
Price per 100 tablets of Product_11: 4.45


In [8]:
#unit wholesale price (this will be the avg cost of each product)
unit_wholesale_price = sales_data.groupby("brand")["cost_"].mean()

for product, cost in unit_wholesale_price.items():
    print(f"Average Wholesale Price of Product_{product}: {cost:.2f}")


Average Wholesale Price of Product_1: 2.18
Average Wholesale Price of Product_2: 3.67
Average Wholesale Price of Product_3: 5.75
Average Wholesale Price of Product_4: 2.03
Average Wholesale Price of Product_5: 3.62
Average Wholesale Price of Product_6: 6.09
Average Wholesale Price of Product_7: 1.85
Average Wholesale Price of Product_8: 2.42
Average Wholesale Price of Product_9: 3.71
Average Wholesale Price of Product_10: 0.91
Average Wholesale Price of Product_11: 1.92


In [9]:
# QUESTION 2
# part a
# To calculate market share Sij, im going to calculate the total market size. 
# Sij will be the number who buy product i in market j /market size (count)
sales_data["market_share"] = sales_data["sales_"] / sales_data["count"]

# outside option share S0j = (any buyer - buyers of headache medicine) / market size
sales_data["outside_share"] = 1 - sales_data.groupby(["store", "week"])["market_share"].transform("sum")

# log market shares (log(s_ij) - log(s_0j))
sales_data["logit_share"] = np.log(sales_data["market_share"]) - np.log(sales_data["outside_share"])

print(sales_data[['store', 'week', 'brand', 'market_share', 'outside_share', 'logit_share']].head())

# dependent variable (logit-transformed market share)
y = sales_data["logit_share"]

# prodcut characteristics price and promotion
X = sales_data[["price_", "prom_"]]  
X = sm.add_constant(X)  

# OLS regression model
ols_model = sm.OLS(y, X).fit()

# Step 4: Print the summary of the OLS regression results
print(ols_model.summary())


   store  week  brand  market_share  outside_share  logit_share
0      2     1      1      0.001128       0.993724    -6.780774
1      2     2      1      0.000859       0.993913    -7.053298
2      2     3      1      0.000443       0.995199    -7.716683
3      2     4      1      0.000874       0.993957    -7.036735
4      2     5      1      0.000728       0.995268    -7.220374
                            OLS Regression Results                            
Dep. Variable:            logit_share   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     315.2
Date:                Fri, 07 Feb 2025   Prob (F-statistic):          1.72e-136
Time:                        22:25:58   Log-Likelihood:                -50711.
No. Observations:               38544   AIC:                         1.014e+05
Df Residuals:                   38541   BIC:                   

In [14]:
# Convert 'product' to numeric (if it's meant to be 1-11)
sales_data["product"] = pd.to_numeric(sales_data["product"], errors="coerce")

# Check if the conversion worked
print(sales_data["product"].dtype)  # Should now be int64 or float64

float64


In [16]:

# Drop NaNs in key columns
# sales_data = sales_data.dropna(subset=["price_", "prom_", "logit_share", "product"])

# Step 1: Create product fixed effects using product column
product_dummies = pd.get_dummies(sales_data["product"], prefix="product", drop_first=True)

# Step 2: Define the dependent variable (logit-transformed market share)
y = sales_data["logit_share"]

# Step 3: Define the independent variables (price, promotion, and product fixed effects)
X = sales_data[["price_", "prom_"]]  # Include price and promotion
X = pd.concat([X, product_dummies], axis=1)  # Add product fixed effects (dummies)
X = sm.add_constant(X)  # Add intercept

# Step 4: Fit the OLS regression model
ols_model = sm.OLS(y, X).fit()

# Step 5: Print the regression results
print(ols_model.summary())


                            OLS Regression Results                            
Dep. Variable:            logit_share   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     315.2
Date:                Fri, 07 Feb 2025   Prob (F-statistic):          1.72e-136
Time:                        22:43:57   Log-Likelihood:                -50711.
No. Observations:               38544   AIC:                         1.014e+05
Df Residuals:                   38541   BIC:                         1.015e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.7660      0.012   -627.345      0.0